In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from pathlib import Path
import glob
import math

def load_images(folder_path):
    """
    지정된 폴더에서 이미지들을 로드하는 함수
    
    Args:
        folder_path (str): 이미지가 있는 폴더 경로
    
    Returns:
        tuple: (로드된 이미지들의 리스트, 이미지 경로들)
    """
    images = []
    image_paths = []
    
    # 지원하는 이미지 확장자
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
    
    for ext in extensions:
        pattern = os.path.join(folder_path, ext)
        image_paths.extend(glob.glob(pattern))
    
    image_paths.sort()  # 정렬하여 일관된 순서 보장
    
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
            print(f"로드된 이미지: {os.path.basename(path)}")
    
    print(f"총 {len(images)}개의 이미지가 로드되었습니다.")
    return images, image_paths

def try_multiple_aruco_dicts():
    """
    다양한 ArUco 딕셔너리를 반환하는 함수
    
    Returns:
        list: ArUco 딕셔너리들의 리스트
    """
    dict_types = [
        cv2.aruco.DICT_5X5_50,
    ]
    
    dict_names = [
        "DICT_5X5_50",
    ]
    
    return dict_types, dict_names

def create_charuco_board(squares_x=14, squares_y=10, square_length=0.04, marker_length=0.02, dict_type=cv2.aruco.DICT_6X6_250):
    """
    ChArUco 보드를 생성하는 함수
    
    Args:
        squares_x (int): 가로 방향 사각형 개수
        squares_y (int): 세로 방향 사각형 개수
        square_length (float): 사각형 크기 (미터)
        marker_length (float): ArUco 마커 크기 (미터)
        dict_type: ArUco 딕셔너리 타입
    
    Returns:
        tuple: (ChArUco 보드 객체, ArUco 딕셔너리)
    """
    aruco_dict = cv2.aruco.getPredefinedDictionary(dict_type)
    
    board = cv2.aruco.CharucoBoard(
        (squares_x, squares_y),
        square_length,
        marker_length,
        aruco_dict
    )
    
    return board, aruco_dict

def detect_aruco_with_multiple_params(gray_img, aruco_dict):
    """
    다양한 파라미터로 ArUco 마커 검출을 시도하는 함수
    
    Args:
        gray_img: 그레이스케일 이미지
        aruco_dict: ArUco 딕셔너리
    
    Returns:
        tuple: (마커 코너들, 마커 ID들, 검출된 파라미터 정보)
    """
    # 기본 파라미터
    parameters = cv2.aruco.DetectorParameters()
    
    # 파라미터 조정 옵션들
    param_sets = [
        # 기본 설정
        {},
        # 더 관대한 설정 1
        {
            'adaptiveThreshWinSizeMin': 3,
            'adaptiveThreshWinSizeMax': 23,
            'adaptiveThreshWinSizeStep': 10,
            'adaptiveThreshConstant': 7
        },
        # 더 관대한 설정 2
        {
            'adaptiveThreshWinSizeMin': 3,
            'adaptiveThreshWinSizeMax': 50,
            'adaptiveThreshWinSizeStep': 4,
            'adaptiveThreshConstant': 7,
            'minMarkerPerimeterRate': 0.03,
            'maxMarkerPerimeterRate': 4.0
        },
        # 매우 관대한 설정
        {
            'adaptiveThreshWinSizeMin': 3,
            'adaptiveThreshWinSizeMax': 100,
            'adaptiveThreshConstant': 5,
            'minMarkerPerimeterRate': 0.01,
            'maxMarkerPerimeterRate': 10.0,
            'polygonalApproxAccuracyRate': 0.1,
            'minCornerDistanceRate': 0.01,
            'minDistanceToBorder': 1
        }
    ]
    
    for i, param_set in enumerate(param_sets):
        # 파라미터 설정
        params = cv2.aruco.DetectorParameters()
        for key, value in param_set.items():
            setattr(params, key, value)
        
        detector = cv2.aruco.ArucoDetector(aruco_dict, params)
        marker_corners, marker_ids, _ = detector.detectMarkers(gray_img)
        
        if len(marker_corners) > 0:
            print(f"파라미터 세트 {i+1}에서 {len(marker_corners)}개 마커 검출")
            return marker_corners, marker_ids, f"파라미터 세트 {i+1}"
    
    return [], None, "검출 실패"

def detect_charuco_with_fallback(images):
    """
    다양한 방법으로 ChArUco 또는 체커보드 검출을 시도하는 함수
    
    Args:
        images: 입력 이미지들
    
    Returns:
        tuple: 검출 결과들
    """
    dict_types, dict_names = try_multiple_aruco_dicts()
    
    # 다양한 보드 크기 시도
    board_sizes = [
        (13, 9),
        # (14, 10), (10, 14), (13, 9), (9, 13), 
        # (12, 8), (8, 12), (11, 7), (7, 11),
        # (10, 6), (6, 10), (9, 6), (6, 9)
    ]
    
    print("다양한 ArUco 딕셔너리와 보드 크기로 검출 시도 중...")
    
    for dict_idx, (dict_type, dict_name) in enumerate(zip(dict_types, dict_names)):
        print(f"\n{dict_name} 딕셔너리 시도 중...")
        
        for board_size in board_sizes:
            try:
                board, aruco_dict = create_charuco_board(
                    squares_x=board_size[0], 
                    squares_y=board_size[1],
                    dict_type=dict_type
                )
                
                all_corners = []
                all_ids = []
                valid_images = []
                detection_info = []
                
                success_count = 0
                
                for i, img in enumerate(images):
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # 다양한 파라미터로 ArUco 마커 검출
                    marker_corners, marker_ids, param_info = detect_aruco_with_multiple_params(gray, aruco_dict)
                    
                    if len(marker_corners) > 0:
                        # ChArUco 코너 검출
                        ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(
                            marker_corners, marker_ids, gray, board
                        )
                        
                        if ret > 0:
                            success_count += 1
                            print(f"  이미지 {i+1}: {len(marker_corners)}개 ArUco 마커, {ret}개 ChArUco 코너 검출 ({param_info})")
                            all_corners.append(charuco_corners)
                            all_ids.append(charuco_ids)
                            valid_images.append(img.copy())
                            
                            detection_info.append({
                                'image_idx': i,
                                'marker_corners': marker_corners,
                                'marker_ids': marker_ids,
                                'charuco_corners': charuco_corners,
                                'charuco_ids': charuco_ids,
                                'num_markers': len(marker_corners),
                                'num_corners': ret,
                                'board_size': board_size,
                                'dict_name': dict_name
                            })
                
                if success_count >= 2:
                    print(f"\n성공! {dict_name}, 보드 크기 {board_size}에서 {success_count}개 이미지 검출")
                    return all_corners, all_ids, valid_images, detection_info, board, aruco_dict
                    
            except Exception as e:
                print(f"  보드 크기 {board_size} 오류: {e}")
                continue
    
    print("\nChArUco 검출 실패. 일반 체커보드 검출로 전환합니다...")
    return detect_regular_checkerboard(images)

def detect_regular_checkerboard(images):
    """
    일반 체커보드 검출을 수행하는 함수 (백업 방법)
    
    Args:
        images: 입력 이미지들
    
    Returns:
        tuple: 검출 결과들
    """
    # 다양한 체커보드 크기 시도
    checkerboard_sizes = [
        (13, 9),
    ]
    
    for size in checkerboard_sizes:
        print(f"체커보드 크기 {size} 시도 중...")
        
        objp = np.zeros((size[0] * size[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:size[0], 0:size[1]].T.reshape(-1, 2)
        
        objpoints = []
        imgpoints = []
        valid_images = []
        detection_info = []
        
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        success_count = 0
        
        for i, img in enumerate(images):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            ret, corners = cv2.findChessboardCorners(gray, size, None)
            
            if ret:
                success_count += 1
                print(f"  이미지 {i+1}에서 체커보드 검출 성공")
                objpoints.append(objp)
                
                corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                imgpoints.append(corners_refined)
                valid_images.append(img.copy())
                
                detection_info.append({
                    'image_idx': i,
                    'corners': corners_refined,
                    'board_size': size,
                    'type': 'regular_checkerboard'
                })
        
        if success_count >= 2:
            print(f"일반 체커보드 검출 성공! 크기 {size}에서 {success_count}개 이미지 검출")
            return objpoints, imgpoints, valid_images, detection_info, None, None
    
    print("모든 검출 방법이 실패했습니다.")
    return None, None, None, None, None, None

def selective_visualization_control():
    """
    시각화 제어를 위한 설정 함수
    
    Returns:
        dict: 시각화 옵션들
    """
    return {
        'show_detection_results': True,     # 검출 결과 표시 여부
        'show_reprojection_errors': True,   # 재투영 오차 표시 여부
        'show_feature_matching': True,      # 특징점 매칭 표시 여부
        'show_3d_camera_view': True,        # 3D 카메라 뷰 표시 여부
        'max_images_per_plot': 6,           # 한 번에 표시할 최대 이미지 수
        'max_matches_display': 50,          # 표시할 최대 매칭 수
        'save_plots': False,                # 플롯을 파일로 저장할지 여부
        'show_first_n_images': None,        # 처음 N개 이미지만 표시 (None이면 모두)
        'max_matching_pairs': 5,            # 최대 매칭 쌍 수
    }

def visualize_detection_results_batch(valid_images, detection_info, max_images_per_plot=6, save_plots=False):
    """
    검출 결과를 배치로 시각화하는 개선된 함수
    
    Args:
        valid_images: 유효한 이미지들
        detection_info: 검출 정보들
        max_images_per_plot: 한 번에 표시할 최대 이미지 수
        save_plots: 플롯을 파일로 저장할지 여부
    """
    total_images = len(valid_images)
    
    if total_images == 0:
        print("시각화할 이미지가 없습니다.")
        return
    
    # 이미지를 배치로 나누기
    num_batches = math.ceil(total_images / max_images_per_plot)
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * max_images_per_plot
        end_idx = min(start_idx + max_images_per_plot, total_images)
        batch_size = end_idx - start_idx
        
        # 동적으로 subplot 구성 계산
        if batch_size <= 2:
            rows, cols = 1, batch_size
        elif batch_size <= 4:
            rows, cols = 2, 2
        elif batch_size <= 6:
            rows, cols = 2, 3
        else:
            rows, cols = 3, 3
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
        if batch_size == 1:
            axes = [axes]
        elif rows == 1 or cols == 1:
            axes = axes.flatten()
        else:
            axes = axes.flatten()
        
        fig.suptitle(f'Detection Results - Batch {batch_idx + 1}/{num_batches}', fontsize=16)
        
        for i, (img, info) in enumerate(zip(valid_images[start_idx:end_idx], 
                                          detection_info[start_idx:end_idx])):
            img_copy = img.copy()
            
            if info.get('type') == 'regular_checkerboard':
                # 일반 체커보드
                cv2.drawChessboardCorners(img_copy, info['board_size'], 
                                        info['corners'], True)
                info_text = f"Checkerboard: {info['board_size']}"
            else:
                # ChArUco
                if info.get('marker_corners') is not None:
                    cv2.aruco.drawDetectedMarkers(img_copy, info['marker_corners'], 
                                                info['marker_ids'])
                
                if info.get('charuco_corners') is not None:
                    cv2.aruco.drawDetectedCornersCharuco(img_copy, info['charuco_corners'], 
                                                      info['charuco_ids'])
                
                info_text = f"ArUco: {info['num_markers']}, ChArUco: {info['num_corners']}"
            
            # 이미지 표시
            axes[i].imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
            axes[i].set_title(f"Image {start_idx + i + 1}\n{info_text}", fontsize=10)
            axes[i].axis('off')
        
        # 사용하지 않는 subplot 숨기기
        for j in range(batch_size, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        
        if save_plots:
            plt.savefig(f'detection_results_batch_{batch_idx + 1}.png', dpi=150, bbox_inches='tight')
        
        plt.show()

def visualize_reprojection_errors_improved(valid_images, poses, corners_or_objpoints, ids_or_imgpoints,
                                         camera_matrix, dist_coeffs, detection_info, board=None, 
                                         detection_type='charuco', max_images_per_plot=6):
    """
    개선된 재투영 오차 시각화 함수
    
    Args:
        valid_images: 유효한 이미지들
        poses: 추정된 포즈들
        corners_or_objpoints: 코너 또는 객체점들
        ids_or_imgpoints: ID 또는 이미지점들  
        camera_matrix: 카메라 행렬
        dist_coeffs: 왜곡 계수
        detection_info: 검출 정보
        board: ChArUco 보드
        detection_type: 검출 타입
        max_images_per_plot: 한 번에 표시할 최대 이미지 수
    """
    # 성공한 포즈들만 필터링
    successful_data = []
    for i, (success, R, t, error) in enumerate(poses):
        if success:
            successful_data.append((i, R, t, error, valid_images[i], detection_info[i]))
    
    if not successful_data:
        print("시각화할 성공한 포즈가 없습니다.")
        return
    
    total_successful = len(successful_data)
    num_batches = math.ceil(total_successful / max_images_per_plot)
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * max_images_per_plot
        end_idx = min(start_idx + max_images_per_plot, total_successful)
        batch_size = end_idx - start_idx
        
        # 동적으로 subplot 구성 계산
        if batch_size <= 2:
            rows, cols = 1, batch_size
        elif batch_size <= 4:
            rows, cols = 2, 2
        elif batch_size <= 6:
            rows, cols = 2, 3
        else:
            rows, cols = 3, 3
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
        if batch_size == 1:
            axes = [axes]
        elif rows == 1 or cols == 1:
            axes = axes.flatten()
        else:
            axes = axes.flatten()
        
        fig.suptitle(f'Reprojection Error Visualization - Batch {batch_idx + 1}/{num_batches}\n'
                    f'(Green: detected points, Red: reprojected points)', fontsize=14)
        
        for plot_idx, data_idx in enumerate(range(start_idx, end_idx)):
            i, R, t, error, img, info = successful_data[data_idx]
            
            img_copy = img.copy()
            
            if detection_type == 'charuco':
                # ChArUco 재투영
                if info['charuco_corners'] is not None:
                    obj_points = board.getChessboardCorners()[info['charuco_ids'].flatten()]
                    
                    rvec, _ = cv2.Rodrigues(R)
                    projected_points, _ = cv2.projectPoints(
                        obj_points, rvec, t, camera_matrix, dist_coeffs
                    )
                    
                    # 원본 점들 (초록색)
                    for point in info['charuco_corners']:
                        cv2.circle(img_copy, tuple(point[0].astype(int)), 5, (0, 255, 0), -1)
                    
                    # 재투영된 점들 (빨간색)
                    for point in projected_points:
                        cv2.circle(img_copy, tuple(point[0].astype(int)), 3, (0, 0, 255), -1)
                    
                    # 오차 선 그리기
                    for orig, proj in zip(info['charuco_corners'], projected_points):
                        cv2.line(img_copy, tuple(orig[0].astype(int)), tuple(proj[0].astype(int)), 
                                (255, 0, 0), 1)
            else:
                # 일반 체커보드 재투영
                board_size = info['board_size']
                objp = np.zeros((board_size[0] * board_size[1], 3), np.float32)
                objp[:, :2] = np.mgrid[0:board_size[0], 0:board_size[1]].T.reshape(-1, 2)
                
                rvec, _ = cv2.Rodrigues(R)
                projected_points, _ = cv2.projectPoints(
                    objp, rvec, t, camera_matrix, dist_coeffs
                )
                
                # 원본 점들 (초록색)
                for point in info['corners']:
                    cv2.circle(img_copy, tuple(point[0].astype(int)), 3, (0, 255, 0), -1)
                
                # 재투영된 점들 (빨간색)  
                for point in projected_points:
                    cv2.circle(img_copy, tuple(point[0].astype(int)), 2, (0, 0, 255), -1)
            
            axes[plot_idx].imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
            axes[plot_idx].set_title(f'Image {i+1}\nReprojection Error: {error:.4f} pixels', fontsize=10)
            axes[plot_idx].axis('off')
        
        # 사용하지 않는 subplot 숨기기
        for j in range(batch_size, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()

def visualize_matches_universal_improved(img1, img2, points1, points2, title="Feature Matching", 
                                       max_matches_display=50):
    """
    개선된 범용 매칭 시각화 함수
    
    Args:
        img1, img2: 입력 이미지들
        points1, points2: 매칭된 특징점들
        title: 플롯 제목
        max_matches_display: 표시할 최대 매칭 수 (너무 많으면 시각화가 복잡해짐)
    """
    # 너무 많은 매칭점이 있으면 샘플링
    if len(points1) > max_matches_display:
        indices = np.random.choice(len(points1), max_matches_display, replace=False)
        points1_display = points1[indices]
        points2_display = points2[indices]
        display_info = f" (showing {max_matches_display}/{len(points1)} matches)"
    else:
        points1_display = points1
        points2_display = points2
        display_info = ""
    
    kp1 = [cv2.KeyPoint(x, y, 1) for x, y in points1_display]
    kp2 = [cv2.KeyPoint(x, y, 1) for x, y in points2_display]
    matches = [cv2.DMatch(i, i, 0) for i in range(len(points1_display))]
    
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None,
                                 flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    plt.figure(figsize=(20, 10))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(f"{title}\nMatched Feature Points: {len(points1)}{display_info}", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def match_features_universal(data1, data2, detection_type):
    """
    범용 특징점 매칭 함수
    """
    if detection_type == 'charuco':
        # ChArUco 매칭
        corners1, ids1 = data1
        corners2, ids2 = data2
        
        ids1_flat = ids1.flatten()
        ids2_flat = ids2.flatten()
        common_ids = np.intersect1d(ids1_flat, ids2_flat)
        
        if len(common_ids) == 0:
            return None, None, None
        
        matched_corners1 = []
        matched_corners2 = []
        
        for common_id in common_ids:
            idx1 = np.where(ids1_flat == common_id)[0]
            idx2 = np.where(ids2_flat == common_id)[0]
            
            if len(idx1) > 0 and len(idx2) > 0:
                matched_corners1.append(corners1[idx1[0]])
                matched_corners2.append(corners2[idx2[0]])
        
        if len(matched_corners1) >= 4:
            matched_corners1 = np.array(matched_corners1).reshape(-1, 2)
            matched_corners2 = np.array(matched_corners2).reshape(-1, 2)
            return matched_corners1, matched_corners2, common_ids
        
    else:
        # 일반 체커보드 매칭
        corners1 = data1.reshape(-1, 2)
        corners2 = data2.reshape(-1, 2)
        return corners1, corners2, None
    
    return None, None, None

def compute_homography_ransac(points1, points2, ransac_threshold=5.0):
    """
    RANSAC을 사용하여 robust한 호모그래피를 계산하는 함수
    """
    if len(points1) >= 4:
        H, mask = cv2.findHomography(points1, points2, 
                                   cv2.RANSAC, 
                                   ransac_threshold)
        return H, mask
    else:
        print("호모그래피 계산을 위해서는 최소 4개의 점이 필요합니다.")
        return None, None

def calibrate_camera_universal(corners_or_objpoints, ids_or_imgpoints, valid_images, 
                              detection_info, board=None, detection_type='charuco'):
    """
    범용 카메라 캘리브레이션 함수
    
    Args:
        corners_or_objpoints: ChArUco 코너 또는 객체점들
        ids_or_imgpoints: ChArUco ID 또는 이미지점들  
        valid_images: 유효한 이미지들
        detection_info: 검출 정보
        board: ChArUco 보드 (ChArUco 모드에서만)
        detection_type: 검출 타입 ('charuco' 또는 'regular_checkerboard')
    
    Returns:
        tuple: (성공여부, 카메라행렬, 왜곡계수, 회전벡터들, 변환벡터들, 재투영오차)
    """
    if len(valid_images) < 3:
        print("카메라 캘리브레이션을 위해서는 최소 3개의 유효한 이미지가 필요합니다.")
        return False, None, None, None, None, None
    
    image_size = (valid_images[0].shape[1], valid_images[0].shape[0])
    print(f"이미지 크기: {image_size}")
    print(f"캘리브레이션에 사용될 이미지 수: {len(valid_images)}")
    
    if detection_type == 'charuco':
        # ChArUco 캘리브레이션
        print("ChArUco 패턴을 사용한 카메라 캘리브레이션 수행 중...")
        ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(
            corners_or_objpoints, ids_or_imgpoints, board, image_size, None, None
        )
    else:
        # 일반 체커보드 캘리브레이션
        print("일반 체커보드 패턴을 사용한 카메라 캘리브레이션 수행 중...")
        ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
            corners_or_objpoints, ids_or_imgpoints, image_size, None, None
        )
    
    return ret, camera_matrix, dist_coeffs, rvecs, tvecs, ret

def estimate_pose_for_each_image(corners_or_objpoints, ids_or_imgpoints, camera_matrix, 
                                dist_coeffs, detection_info, board=None, detection_type='charuco'):
    """
    각 이미지에 대해 개별적으로 외부 파라미터를 추정하는 함수
    
    Args:
        corners_or_objpoints: 코너 또는 객체점들
        ids_or_imgpoints: ID 또는 이미지점들
        camera_matrix: 카메라 내부 파라미터 행렬
        dist_coeffs: 왜곡 계수
        detection_info: 검출 정보
        board: ChArUco 보드
        detection_type: 검출 타입
    
    Returns:
        list: 각 이미지의 포즈 정보 [(성공여부, R, t, 재투영오차), ...]
    """
    poses = []
    
    for i, info in enumerate(detection_info):
        print(f"\n이미지 {i+1}의 포즈 추정 중...")
        
        if detection_type == 'charuco':
            # ChArUco에서 3D-2D 대응점 생성
            if info['charuco_corners'] is not None and len(info['charuco_corners']) >= 4:
                # 보드에서 3D 좌표 얻기
                obj_points = board.getChessboardCorners()[info['charuco_ids'].flatten()]
                img_points = info['charuco_corners']
                
                # PnP 문제 해결
                success, rvec, tvec = cv2.solvePnP(
                    obj_points, img_points, camera_matrix, dist_coeffs
                )
                
                if success:
                    # 회전 벡터를 회전 행렬로 변환
                    R, _ = cv2.Rodrigues(rvec)
                    
                    # 재투영 오차 계산
                    projected_points, _ = cv2.projectPoints(
                        obj_points, rvec, tvec, camera_matrix, dist_coeffs
                    )
                    reprojection_error = np.mean(np.linalg.norm(
                        img_points.reshape(-1, 2) - projected_points.reshape(-1, 2), axis=1
                    ))
                    
                    poses.append((True, R, tvec, reprojection_error))
                    print(f"  포즈 추정 성공, 재투영 오차: {reprojection_error:.4f} 픽셀")
                else:
                    poses.append((False, None, None, None))
                    print("  포즈 추정 실패")
            else:
                poses.append((False, None, None, None))
                print("  충분한 특징점이 없음")
        
        else:
            # 일반 체커보드에서 포즈 추정
            board_size = info['board_size']
            
            # 3D 객체점 생성
            objp = np.zeros((board_size[0] * board_size[1], 3), np.float32)
            objp[:, :2] = np.mgrid[0:board_size[0], 0:board_size[1]].T.reshape(-1, 2)
            
            img_points = info['corners']
            
            # PnP 문제 해결
            success, rvec, tvec = cv2.solvePnP(
                objp, img_points, camera_matrix, dist_coeffs
            )
            
            if success:
                # 회전 벡터를 회전 행렬로 변환
                R, _ = cv2.Rodrigues(rvec)
                
                # 재투영 오차 계산
                projected_points, _ = cv2.projectPoints(
                    objp, rvec, tvec, camera_matrix, dist_coeffs
                )
                reprojection_error = np.mean(np.linalg.norm(
                    img_points.reshape(-1, 2) - projected_points.reshape(-1, 2), axis=1
                ))
                
                poses.append((True, R, tvec, reprojection_error))
                print(f"  포즈 추정 성공, 재투영 오차: {reprojection_error:.4f} 픽셀")
            else:
                poses.append((False, None, None, None))
                print("  포즈 추정 실패")
    
    return poses

def analyze_camera_parameters(camera_matrix, dist_coeffs, image_size):
    """
    카메라 파라미터를 분석하는 함수
    
    Args:
        camera_matrix: 카메라 내부 파라미터 행렬
        dist_coeffs: 왜곡 계수
        image_size: 이미지 크기
    
    Returns:
        dict: 분석된 파라미터들
    """
    fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
    cx, cy = camera_matrix[0, 2], camera_matrix[1, 2]
    
    # 화각 계산 (라디안 -> 도)
    fov_x = 2 * np.arctan(image_size[0] / (2 * fx)) * 180 / np.pi
    fov_y = 2 * np.arctan(image_size[1] / (2 * fy)) * 180 / np.pi
    
    # 픽셀 크기 추정 (일반적인 센서 크기 가정)
    sensor_width_mm = 36  # 35mm 풀프레임 가정
    pixel_size_um = (sensor_width_mm * 1000) / image_size[0]
    
    analysis = {
        'focal_length_x': fx,
        'focal_length_y': fy,
        'focal_length_avg': (fx + fy) / 2,
        'principal_point': (cx, cy),
        'image_center': (image_size[0]/2, image_size[1]/2),
        'principal_point_offset': (cx - image_size[0]/2, cy - image_size[1]/2),
        'field_of_view_x': fov_x,
        'field_of_view_y': fov_y,
        'aspect_ratio': fx / fy,
        'pixel_size_estimate_um': pixel_size_um,
        'distortion_coefficients': dist_coeffs.flatten()
    }
    
    return analysis

def print_calibration_results(ret, camera_matrix, dist_coeffs, poses, analysis):
    """
    캘리브레이션 결과를 출력하는 함수
    """
    print("\n" + "="*60)
    print("카메라 캘리브레이션 결과")
    print("="*60)
    
    if ret:
        print(f"✓ 캘리브레이션 성공!")
        print(f"✓ 전체 재투영 오차 (RMS): {ret:.4f} 픽셀")
        
        print(f"\n📷 카메라 내부 파라미터 행렬 (K):")
        print(f"   [{camera_matrix[0,0]:8.2f}  {camera_matrix[0,1]:8.2f}  {camera_matrix[0,2]:8.2f}]")
        print(f"   [{camera_matrix[1,0]:8.2f}  {camera_matrix[1,1]:8.2f}  {camera_matrix[1,2]:8.2f}]")
        print(f"   [{camera_matrix[2,0]:8.2f}  {camera_matrix[2,1]:8.2f}  {camera_matrix[2,2]:8.2f}]")
        
        print(f"\n🔍 상세 파라미터 분석:")
        print(f"   • 초점거리 (fx, fy): ({analysis['focal_length_x']:.2f}, {analysis['focal_length_y']:.2f}) 픽셀")
        print(f"   • 평균 초점거리: {analysis['focal_length_avg']:.2f} 픽셀")
        print(f"   • 주점 (cx, cy): ({analysis['principal_point'][0]:.2f}, {analysis['principal_point'][1]:.2f})")
        print(f"   • 이미지 중심에서 오프셋: ({analysis['principal_point_offset'][0]:.2f}, {analysis['principal_point_offset'][1]:.2f})")
        print(f"   • 화각 (FOV): {analysis['field_of_view_x']:.1f}° × {analysis['field_of_view_y']:.1f}°")
        print(f"   • 종횡비 (fx/fy): {analysis['aspect_ratio']:.4f}")
        print(f"   • 추정 픽셀 크기: {analysis['pixel_size_estimate_um']:.2f} μm")
        
        print(f"\n🌊 왜곡 계수:")
        dist_names = ['k1', 'k2', 'p1', 'p2', 'k3']
        for i, (name, coeff) in enumerate(zip(dist_names, dist_coeffs.flatten())):
            if i < len(dist_coeffs.flatten()):
                print(f"   • {name}: {coeff:.6f}")
    else:
        print("❌ 캘리브레이션 실패")
    
    print("="*60)

# 3D 시각화 관련 함수들
def plot_camera_views(poses, detection_info, board=None, detection_type='charuco', 
                     camera_matrix=None, scale_factor=0.1):
    """
    3D 공간에서 카메라 뷰들을 시각화하는 함수
    """
    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # 성공한 포즈들만 추출
    successful_poses = []
    successful_indices = []
    
    for i, pose_data in enumerate(poses):
        # poses 구조 안전하게 처리
        if isinstance(pose_data, (list, tuple)) and len(pose_data) >= 3:
            if len(pose_data) >= 4:
                success, R, t, error = pose_data
            else:
                success, R, t = pose_data
                error = 0.0  # 기본값
            
            if success and R is not None and t is not None:
                successful_poses.append((R, t, error))
                successful_indices.append(i)
    
    if len(successful_poses) == 0:
        print("시각화할 수 있는 성공한 포즈가 없습니다.")
        return
    
    print(f"\n{len(successful_poses)}개의 카메라 포즈를 시각화합니다...")
    
    # 체커보드/ChArUco 패턴 시각화
    if detection_type == 'charuco' and board is not None:
        plot_charuco_board_3d(ax, board, scale_factor)
    else:
        # 일반 체커보드의 경우 첫 번째 이미지의 보드 크기 사용
        if len(detection_info) > 0 and successful_indices:
            board_size = detection_info[successful_indices[0]]['board_size']
            plot_checkerboard_3d(ax, board_size, scale_factor)
    
    # 각 카메라 포즈 시각화
    camera_positions = []
    
    for i, (R, t, error) in enumerate(successful_poses):
        # 카메라 중심 위치 계산: C = -R^T * t
        camera_center = -R.T @ t
        camera_positions.append(camera_center.flatten())
        
        # 카메라 시각화
        plot_camera_3d(ax, R, camera_center.flatten(), scale_factor, 
                      f'Camera {successful_indices[i]+1}', error)
    
    # 카메라 궤적 그리기 (카메라가 3개 이상인 경우)
    if len(camera_positions) >= 3:
        camera_positions = np.array(camera_positions)
        ax.plot(camera_positions[:, 0], camera_positions[:, 1], camera_positions[:, 2], 
                'b--', alpha=0.6, linewidth=2, label='Camera Trajectory')
    
    # 좌표축 설정 및 레이블
    ax.set_xlabel('X (m)', fontsize=12)
    ax.set_ylabel('Y (m)', fontsize=12)
    ax.set_zlabel('Z (m)', fontsize=12)
    ax.set_title('3D Camera Views and Checkerboard Layout', fontsize=14, pad=20)
    
    # 범례 추가
    ax.legend(loc='upper right')
    
    # 축 범위 자동 조정
    if len(camera_positions) > 0:
        all_points = np.array(camera_positions)
        center = np.mean(all_points, axis=0)
        max_range = np.max(np.abs(all_points - center)) * 1.2
        
        ax.set_xlim(center[0] - max_range, center[0] + max_range)
        ax.set_ylim(center[1] - max_range, center[1] + max_range)
        ax.set_zlim(center[2] - max_range, center[2] + max_range)
    
    # 격자 표시
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 카메라 정보 요약 출력
    print_camera_summary(successful_poses, successful_indices, camera_positions)

def plot_camera_3d(ax, R, camera_center, scale, label, error):
    """
    3D 공간에서 개별 카메라를 시각화하는 함수
    """
    # 카메라 좌표계 축 벡터
    axes_length = scale
    
    # 카메라 좌표계 (X=빨강, Y=초록, Z=파랑)
    x_axis = R[:, 0] * axes_length + camera_center
    y_axis = R[:, 1] * axes_length + camera_center  
    z_axis = R[:, 2] * axes_length + camera_center
    
    # 좌표축 그리기
    ax.plot([camera_center[0], x_axis[0]], [camera_center[1], x_axis[1]], 
            [camera_center[2], x_axis[2]], 'r-', linewidth=3, alpha=0.8)
    ax.plot([camera_center[0], y_axis[0]], [camera_center[1], y_axis[1]], 
            [camera_center[2], y_axis[2]], 'g-', linewidth=3, alpha=0.8)
    ax.plot([camera_center[0], z_axis[0]], [camera_center[1], z_axis[1]], 
            [camera_center[2], z_axis[2]], 'b-', linewidth=3, alpha=0.8)
    
    # 카메라 중심점 표시
    ax.scatter(*camera_center, s=100, c='black', marker='o', alpha=0.8)
    
    # 카메라 프러스텀 (시야각) 표시
    if scale > 0:
        plot_camera_frustum(ax, R, camera_center, scale)
    
    # 레이블 추가
    ax.text(camera_center[0], camera_center[1], camera_center[2] + scale*0.5, 
            f'{label}\nError: {error:.3f}px', fontsize=9, ha='center')

def plot_camera_frustum(ax, R, camera_center, scale):
    """
    카메라 프러스텀(시야각)을 시각화하는 함수
    """
    # 프러스텀 크기
    frustum_length = scale * 1.5
    frustum_width = scale * 0.8
    frustum_height = scale * 0.6
    
    # 프러스텀 모서리 점들 (카메라 좌표계에서)
    frustum_points_local = np.array([
        [0, 0, 0],  # 카메라 중심
        [-frustum_width, -frustum_height, frustum_length],  # 좌하
        [frustum_width, -frustum_height, frustum_length],   # 우하
        [frustum_width, frustum_height, frustum_length],    # 우상
        [-frustum_width, frustum_height, frustum_length]    # 좌상
    ])
    
    # 월드 좌표계로 변환
    frustum_points_world = []
    for point in frustum_points_local:
        world_point = R @ point + camera_center
        frustum_points_world.append(world_point)
    
    frustum_points_world = np.array(frustum_points_world)
    
    # 프러스텀 선 그리기
    # 카메라 중심에서 각 모서리로
    for i in range(1, 5):
        ax.plot([frustum_points_world[0, 0], frustum_points_world[i, 0]],
                [frustum_points_world[0, 1], frustum_points_world[i, 1]],
                [frustum_points_world[0, 2], frustum_points_world[i, 2]], 
                'gray', alpha=0.4, linewidth=1)
    
    # 프러스텀 사각형 그리기
    for i in range(1, 5):
        next_i = i + 1 if i < 4 else 1
        ax.plot([frustum_points_world[i, 0], frustum_points_world[next_i, 0]],
                [frustum_points_world[i, 1], frustum_points_world[next_i, 1]],
                [frustum_points_world[i, 2], frustum_points_world[next_i, 2]], 
                'gray', alpha=0.4, linewidth=1)

def plot_charuco_board_3d(ax, board, scale):
    """
    3D 공간에서 ChArUco 보드를 시각화하는 함수
    """
    # ChArUco 보드의 3D 코너 점들 가져오기
    board_corners = board.getChessboardCorners()
    
    if len(board_corners) > 0:
        # 보드 평면 (Z=0)에 점들 표시
        x_coords = board_corners[:, 0]
        y_coords = board_corners[:, 1]
        z_coords = board_corners[:, 2]
        
        # 코너 점들 표시
        ax.scatter(x_coords, y_coords, z_coords, c='red', s=20, alpha=0.7, label='ChArUco Corners')
        
        # 보드 경계 그리기
        board_size = board.getChessboardSize()
        square_size = board.getSquareLength()
        
        # 보드 경계 계산
        max_x = (board_size[0] - 1) * square_size
        max_y = (board_size[1] - 1) * square_size
        
        # 경계선 그리기
        boundary_x = [0, max_x, max_x, 0, 0]
        boundary_y = [0, 0, max_y, max_y, 0]
        boundary_z = [0, 0, 0, 0, 0]
        
        ax.plot(boundary_x, boundary_y, boundary_z, 'k-', linewidth=2, alpha=0.8, label='Board Boundary')

def plot_checkerboard_3d(ax, board_size, scale):
    """
    3D 공간에서 일반 체커보드를 시각화하는 함수
    """
    # 체커보드 코너 점들 생성 (Z=0 평면)
    square_size = scale  # 정사각형 크기
    
    x_coords = []
    y_coords = []
    z_coords = []
    
    for i in range(board_size[0]):
        for j in range(board_size[1]):
            x_coords.append(i * square_size)
            y_coords.append(j * square_size)
            z_coords.append(0)
    
    # 코너 점들 표시
    ax.scatter(x_coords, y_coords, z_coords, c='red', s=20, alpha=0.7, label='Checkerboard Corners')
    
    # 보드 경계 그리기
    max_x = (board_size[0] - 1) * square_size
    max_y = (board_size[1] - 1) * square_size
    
    boundary_x = [0, max_x, max_x, 0, 0]
    boundary_y = [0, 0, max_y, max_y, 0]
    boundary_z = [0, 0, 0, 0, 0]
    
    ax.plot(boundary_x, boundary_y, boundary_z, 'k-', linewidth=2, alpha=0.8, label='Board Boundary')

def print_camera_summary(successful_poses, successful_indices, camera_positions):
    """
    카메라 정보 요약을 출력하는 함수
    """
    print(f"\n📷 카메라 궤적 분석 요약:")
    print(f"=" * 50)
    
    camera_positions = np.array(camera_positions)
    
    # 전체 통계
    if len(camera_positions) > 1:
        total_distance = 0
        for i in range(1, len(camera_positions)):
            total_distance += np.linalg.norm(camera_positions[i] - camera_positions[i-1])
        
        workspace_size = np.max(camera_positions, axis=0) - np.min(camera_positions, axis=0)
        
        print(f"🎯 전체 통계:")
        print(f"   • 총 이동 거리: {total_distance:.3f} m")
        print(f"   • 작업 공간 크기: {workspace_size[0]:.3f} × {workspace_size[1]:.3f} × {workspace_size[2]:.3f} m")
        print(f"   • 평균 높이: {np.mean(camera_positions[:, 2]):.3f} m")
        print(f"   • 높이 범위: {np.min(camera_positions[:, 2]):.3f} ~ {np.max(camera_positions[:, 2]):.3f} m")

def plot_comprehensive_camera_view(calibration_results):
    """
    포괄적인 카메라 뷰 시각화 함수
    """
    poses = calibration_results['poses']
    detection_info = calibration_results['detection_info'] 
    detection_type = calibration_results['detection_type']
    camera_matrix = calibration_results['camera_matrix']
    
    # board 정보 추출
    board = None
    if detection_type == 'charuco':
        # ChArUco 보드 재생성 (detection_info에서 정보 추출)
        if len(detection_info) > 0:
            # 첫 번째 성공한 검출 정보 사용
            for info in detection_info:
                if 'board_size' in info:
                    board, _ = create_charuco_board(
                        squares_x=info['board_size'][0], 
                        squares_y=info['board_size'][1]
                    )
                    break
    
    print(f"\n🎬 3D 카메라 뷰 시각화 시작...")
    
    # 1. 메인 3D 뷰
    plot_camera_views(poses, detection_info, board, detection_type, camera_matrix)
    
    return True

# === 개선된 메인 실행 코드 ===
def main():
    folder = "./camera_images2"  # 폴더 경로 설정
    
    # 시각화 옵션 설정
    viz_options = selective_visualization_control()
    
    # 폴더가 존재하지 않는 경우 현재 디렉토리에서 이미지 찾기
    if not os.path.exists(folder):
        print(f"폴더 '{folder}'가 존재하지 않습니다. 현재 디렉토리에서 이미지를 찾습니다.")
        folder = "."
    
    # 이미지 로드
    images, image_paths = load_images(folder)
    
    if len(images) < 2:
        print("최소 2개의 이미지가 필요합니다.")
        return
    
    # 이미지 수가 많을 때 제한 옵션
    if viz_options['show_first_n_images'] and len(images) > viz_options['show_first_n_images']:
        print(f"이미지가 {len(images)}개로 많아서 처음 {viz_options['show_first_n_images']}개만 처리합니다.")
        images = images[:viz_options['show_first_n_images']]
        image_paths = image_paths[:viz_options['show_first_n_images']]
    
    # 특징점 검출 (ChArUco 또는 체커보드)
    print("\n특징점 검출 시작...")
    result = detect_charuco_with_fallback(images)
    
    if result[0] is None:
        print("모든 검출 방법이 실패했습니다.")
        return
    
    corners_or_objpoints, ids_or_imgpoints, valid_images, detection_info, board, aruco_dict = result
    
    # 검출 타입 확인
    detection_type = 'charuco' if board is not None else 'regular_checkerboard'
    print(f"\n검출 타입: {detection_type}")
    
    # 검출 결과 시각화 (개선된 버전)
    if viz_options['show_detection_results']:
        print("\n검출 결과 시각화...")
        visualize_detection_results_batch(
            valid_images, 
            detection_info, 
            max_images_per_plot=viz_options['max_images_per_plot'],
            save_plots=viz_options['save_plots']
        )
    
    # ==========================================
    # 카메라 캘리브레이션 수행
    # ==========================================
    print("\n" + "="*50)
    print("카메라 캘리브레이션 시작")
    print("="*50)
    
    ret, camera_matrix, dist_coeffs, rvecs, tvecs, reprojection_error = calibrate_camera_universal(
        corners_or_objpoints, ids_or_imgpoints, valid_images, detection_info, board, detection_type
    )
    
    if not ret:
        print("카메라 캘리브레이션에 실패했습니다.")
        return
    
    # 카메라 파라미터 분석
    image_size = (valid_images[0].shape[1], valid_images[0].shape[0])
    analysis = analyze_camera_parameters(camera_matrix, dist_coeffs, image_size)
    
    # ==========================================
    # 각 이미지별 포즈 추정
    # ==========================================
    print("\n각 이미지별 외부 파라미터 추정 중...")
    poses = estimate_pose_for_each_image(
        corners_or_objpoints, ids_or_imgpoints, camera_matrix, dist_coeffs, 
        detection_info, board, detection_type
    )
    
    # ==========================================
    # 결과 출력 및 시각화
    # ==========================================
    
    # 상세한 캘리브레이션 결과 출력
    print_calibration_results(ret, camera_matrix, dist_coeffs, poses, analysis)
    
    # 재투영 오차 시각화 (개선된 버전)
    if viz_options['show_reprojection_errors']:
        print("\n재투영 오차 시각화 생성 중...")
        visualize_reprojection_errors_improved(
            valid_images, poses, corners_or_objpoints, ids_or_imgpoints,
            camera_matrix, dist_coeffs, detection_info, board, detection_type,
            max_images_per_plot=viz_options['max_images_per_plot']
        )
    
    # ==========================================
    # 특징점 매칭 및 호모그래피 계산 (추가 분석용)
    # ==========================================
    homographies = []
    
    if viz_options['show_feature_matching']:
        print("\n특징점 매칭 및 호모그래피 계산...")
        # 너무 많은 매칭 시각화를 방지하기 위해 최대 5개 쌍만 표시
        max_pairs_to_show = min(viz_options['max_matching_pairs'], len(valid_images) - 1)
        
        for i in range(1, min(len(valid_images), max_pairs_to_show + 1)):
            print(f"\n이미지 1과 이미지 {i+1} 간의 매칭...")
            
            if detection_type == 'charuco':
                data1 = (corners_or_objpoints[0], ids_or_imgpoints[0])
                data2 = (corners_or_objpoints[i], ids_or_imgpoints[i])
            else:
                data1 = ids_or_imgpoints[0]  # imgpoints for regular checkerboard
                data2 = ids_or_imgpoints[i]
            
            matched_points1, matched_points2, common_data = match_features_universal(
                data1, data2, detection_type
            )
            
            if matched_points1 is not None:
                print(f"매칭된 특징점 개수: {len(matched_points1)}")
                
                # 매칭 결과 시각화 (개선된 버전)
                visualize_matches_universal_improved(
                    valid_images[0], valid_images[i],
                    matched_points1, matched_points2,
                    f"Feature Point Matching: Image 1 vs Image {i+1}",
                    max_matches_display=viz_options['max_matches_display']
                )
                
                # 호모그래피 계산
                H, mask = compute_homography_ransac(matched_points1, matched_points2)
                if H is not None:
                    det_H = np.linalg.det(H)
                    inlier_ratio = np.sum(mask) / len(mask)
                    print(f"호모그래피 행렬 {i}:")
                    print(H)
                    print(f'det H: {det_H:.6f}')
                    print(f"인라이어 비율: {inlier_ratio:.2%}")
                    print(f"인라이어 개수: {np.sum(mask)}/{len(mask)}")
                    homographies.append(H)
            else:
                print("충분한 매칭점을 찾을 수 없습니다.")
        
        # 나머지 이미지들에 대해서는 호모그래피만 계산 (시각화 없이)
        for i in range(max_pairs_to_show + 1, len(valid_images)):
            if detection_type == 'charuco':
                data1 = (corners_or_objpoints[0], ids_or_imgpoints[0])
                data2 = (corners_or_objpoints[i], ids_or_imgpoints[i])
            else:
                data1 = ids_or_imgpoints[0]
                data2 = ids_or_imgpoints[i]
            
            matched_points1, matched_points2, common_data = match_features_universal(
                data1, data2, detection_type
            )
            
            if matched_points1 is not None:
                H, mask = compute_homography_ransac(matched_points1, matched_points2)
                if H is not None:
                    homographies.append(H)
    
    print(f"\n총 {len(homographies)}개의 호모그래피 행렬이 계산되었습니다.")
    
    # ==========================================
    # 최종 결과 정리
    # ==========================================
    calibration_results = {
        'camera_matrix': camera_matrix,
        'distortion_coefficients': dist_coeffs,
        'reprojection_error': ret,
        'poses': poses,
        'analysis': analysis,
        'homographies': homographies,
        'detection_type': detection_type,
        'valid_images': valid_images,
        'detection_info': detection_info
    }
    
    print(f"\n🎉 캘리브레이션 완료!")
    print(f"📊 사용된 이미지: {len(valid_images)}개")
    print(f"📷 검출 방법: {detection_type}")
    print(f"🎯 전체 재투영 오차: {ret:.4f} 픽셀")
    
    # ==========================================
    # 3D 카메라 뷰 시각화
    # ==========================================
    if viz_options['show_3d_camera_view']:
        print(f"\n🎬 3D 카메라 뷰 생성 중...")
        plot_comprehensive_camera_view(calibration_results)
    
    return calibration_results

# 실행
if __name__ == "__main__":
    calibration_results = main()
    
    if calibration_results:
        # 결과에 쉽게 접근할 수 있도록 변수 할당
        K = calibration_results['camera_matrix']  # 카메라 내부 파라미터 행렬
        dist_coeffs = calibration_results['distortion_coefficients']  # 왜곡 계수
        poses = calibration_results['poses']  # 각 이미지의 [R,t]
        homographies = calibration_results['homographies']  # 호모그래피 행렬들
        
        print(f"\n🔍 주요 결과 변수들:")
        print(f"   • K (카메라 내부 파라미터): {K.shape} 행렬")
        print(f"   • dist_coeffs (왜곡 계수): {dist_coeffs.shape} 벡터") 
        print(f"   • poses (외부 파라미터): {len(poses)}개 이미지의 [R,t]")
        print(f"   • homographies: {len(homographies)}개 호모그래피 행렬")
        
        print(f"\n💡 사용 예시:")
        print(f"   • 카메라 행렬 접근: K = calibration_results['camera_matrix']")
        print(f"   • 첫 번째 이미지 회전 행렬: R1 = poses[0][1]")
        print(f"   • 첫 번째 이미지 변환 벡터: t1 = poses[0][2]")
        print(f"   • 첫 번째 호모그래피: H1 = homographies[0]")
        
        print(f"\n🎬 3D 시각화 정보:")
        print(f"   • 빨간색 축: X축 (카메라 우측)")
        print(f"   • 초록색 축: Y축 (카메라 아래)")  
        print(f"   • 파란색 축: Z축 (카메라 전방)")
        print(f"   • 회색 선: 카메라 시야각(프러스텀)")
        print(f"   • 빨간 점: 체커보드 코너점들")
        print(f"   • 검은 선: 체커보드 경계")
    else:
        print("캘리브레이션이 실패했습니다.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from mpl_toolkits.mplot3d import Axes3D

def validate_calibration_comprehensive(calibration_results):
    """
    캘리브레이션 품질을 종합적으로 검증하는 함수
    
    Args:
        calibration_results: 이전 셀에서 생성된 캘리브레이션 결과
    
    Returns:
        dict: 검증 결과 및 품질 평가
    """
    print("="*60)
    print("COMPREHENSIVE CALIBRATION QUALITY VALIDATION")
    print("="*60)
    
    # 결과 추출
    camera_matrix = calibration_results['camera_matrix']
    dist_coeffs = calibration_results['distortion_coefficients']
    poses = calibration_results['poses']
    homographies = calibration_results['homographies']
    reprojection_error = calibration_results['reprojection_error']
    valid_images = calibration_results['valid_images']
    detection_info = calibration_results['detection_info']
    
    validation_results = {'overall_score': 0, 'max_score': 25}
    
    # 1. 재투영 오차 분석
    print("\n1. REPROJECTION ERROR ANALYSIS")
    print("-" * 40)
    
    individual_errors = [pose[3] for pose in poses if pose[0] and len(pose) > 3 and pose[3] is not None]
    
    if individual_errors:
        mean_error = np.mean(individual_errors)
        std_error = np.std(individual_errors)
        max_error = np.max(individual_errors)
        min_error = np.min(individual_errors)
        
        print(f"Overall RMS error: {reprojection_error:.4f} pixels")
        print(f"Individual errors: {min_error:.4f} - {max_error:.4f} pixels")
        print(f"Mean ± Std: {mean_error:.4f} ± {std_error:.4f} pixels")
        
        # 재투영 오차 점수화 (5점 만점)
        if reprojection_error < 0.5:
            error_score = 5
            error_grade = "EXCELLENT"
        elif reprojection_error < 1.0:
            error_score = 4  
            error_grade = "GOOD"
        elif reprojection_error < 2.0:
            error_score = 3
            error_grade = "ACCEPTABLE"
        elif reprojection_error < 5.0:
            error_score = 2
            error_grade = "POOR"
        else:
            error_score = 1
            error_grade = "VERY POOR"
        
        print(f"Grade: {error_grade} ({error_score}/5 points)")
        validation_results['reprojection'] = {
            'rms_error': reprojection_error,
            'grade': error_grade,
            'score': error_score
        }
        validation_results['overall_score'] += error_score
    
    # 2. 카메라 내부 파라미터 검증
    print("\n2. INTRINSIC PARAMETERS VALIDATION")
    print("-" * 40)
    
    fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
    cx, cy = camera_matrix[0, 2], camera_matrix[1, 2]
    
    if valid_images:
        img_height, img_width = valid_images[0].shape[:2]
        
        # 주점 오프셋 검사
        center_offset_x = abs(cx - img_width/2) / img_width
        center_offset_y = abs(cy - img_height/2) / img_height
        max_center_offset = max(center_offset_x, center_offset_y)
        
        # 종횡비 검사
        aspect_ratio = fx / fy
        aspect_deviation = abs(aspect_ratio - 1.0)
        
        # 초점거리 합리성 검사
        diagonal_pixels = np.sqrt(img_width**2 + img_height**2)
        avg_focal_length = (fx + fy) / 2
        focal_ratio = avg_focal_length / diagonal_pixels
        
        print(f"Image size: {img_width} × {img_height}")
        print(f"Focal lengths: fx={fx:.1f}, fy={fy:.1f}")
        print(f"Principal point: ({cx:.1f}, {cy:.1f})")
        print(f"Principal point offset: {max_center_offset:.2%}")
        print(f"Aspect ratio (fx/fy): {aspect_ratio:.4f}")
        print(f"Focal length ratio: {focal_ratio:.3f}")
        
        # 내부 파라미터 점수화 (5점 만점)
        intrinsic_score = 5
        issues = []
        
        if max_center_offset > 0.1:  # 10% 이상 벗어남
            intrinsic_score -= 1
            issues.append("Principal point off-center")
        
        if aspect_deviation > 0.05:  # 5% 이상 차이
            intrinsic_score -= 1
            issues.append("Significant aspect ratio deviation")
        
        if focal_ratio < 0.5 or focal_ratio > 3.0:  # 비정상적 범위
            intrinsic_score -= 2
            issues.append("Unusual focal length")
        
        if not issues:
            print("✓ All intrinsic parameters look reasonable")
        else:
            print("⚠ Issues detected:")
            for issue in issues:
                print(f"  - {issue}")
        
        print(f"Intrinsic score: {intrinsic_score}/5 points")
        validation_results['intrinsics'] = {
            'score': intrinsic_score,
            'issues': issues
        }
        validation_results['overall_score'] += intrinsic_score
    
    # 3. 호모그래피 검증
    print("\n3. HOMOGRAPHY VALIDATION")
    print("-" * 40)
    
    homography_score = 0
    if homographies:
        det_values = []
        cond_values = []
        
        for i, H in enumerate(homographies):
            det_H = np.linalg.det(H)
            cond_H = np.linalg.cond(H)
            
            det_values.append(det_H)
            cond_values.append(cond_H)
            
            print(f"Homography {i+1}: det={det_H:.4f}, cond={cond_H:.1f}")
        
        # 호모그래피 품질 평가 (5점 만점)
        positive_dets = sum(1 for det in det_values if det > 0)
        good_condition = sum(1 for cond in cond_values if cond < 100)
        
        det_ratio = positive_dets / len(det_values) if det_values else 0
        cond_ratio = good_condition / len(cond_values) if cond_values else 0
        
        homography_score = int(2.5 * det_ratio + 2.5 * cond_ratio)
        
        print(f"Positive determinants: {positive_dets}/{len(det_values)}")
        print(f"Good condition numbers: {good_condition}/{len(cond_values)}")
        print(f"Homography score: {homography_score}/5 points")
        
        validation_results['homography'] = {
            'score': homography_score,
            'determinants': det_values,
            'condition_numbers': cond_values
        }
    
    validation_results['overall_score'] += homography_score
    
    # 4. 외부 파라미터 검증
    print("\n4. EXTRINSIC PARAMETERS VALIDATION")
    print("-" * 40)
    
    extrinsic_score = 0
    successful_poses = [pose for pose in poses if pose[0]]
    
    if len(successful_poses) >= 2:
        # 카메라 위치 추출
        camera_positions = []
        rotation_angles = []
        
        for success, R, t, error in successful_poses:
            if success and R is not None and t is not None:
                camera_center = (-R.T @ t).flatten()
                camera_positions.append(camera_center)
                
                # 오일러 각도 계산
                sy = np.sqrt(R[0,0] * R[0,0] + R[1,0] * R[1,0])
                if sy > 1e-6:
                    x = np.arctan2(R[2,1], R[2,2])
                    y = np.arctan2(-R[2,0], sy)
                    z = np.arctan2(R[1,0], R[0,0])
                else:
                    x = np.arctan2(-R[1,2], R[1,1])
                    y = np.arctan2(-R[2,0], sy)
                    z = 0
                
                rotation_angles.append([np.degrees(x), np.degrees(y), np.degrees(z)])
        
        camera_positions = np.array(camera_positions)
        rotation_angles = np.array(rotation_angles)
        
        # 위치 일관성 검사
        if len(camera_positions) > 1:
            distances = []
            for i in range(1, len(camera_positions)):
                dist = np.linalg.norm(camera_positions[i] - camera_positions[i-1])
                distances.append(dist)
            
            distance_std = np.std(distances) if distances else 0
            mean_distance = np.mean(distances) if distances else 0
            
            print(f"Camera positions:")
            for i, pos in enumerate(camera_positions):
                print(f"  Camera {i+1}: ({pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f})")
            
            print(f"Inter-camera distances: {mean_distance:.3f} ± {distance_std:.3f} m")
            
            # 회전 일관성 검사
            if len(rotation_angles) > 1:
                rotation_std = np.std(rotation_angles, axis=0)
                print(f"Rotation variations: Roll={rotation_std[0]:.1f}°, Pitch={rotation_std[1]:.1f}°, Yaw={rotation_std[2]:.1f}°")
            
            # 외부 파라미터 점수화 (5점 만점)
            extrinsic_score = 5
            
            if distance_std / mean_distance > 0.5 if mean_distance > 0 else False:  # 거리 편차가 50% 이상
                extrinsic_score -= 1
                print("⚠ High variation in camera distances")
            
            if np.any(rotation_std > 45):  # 회전 편차가 45도 이상
                extrinsic_score -= 1
                print("⚠ High variation in camera orientations")
            
            print(f"Extrinsic score: {extrinsic_score}/5 points")
    
    validation_results['extrinsics'] = {'score': extrinsic_score}
    validation_results['overall_score'] += extrinsic_score
    
    # 5. 기하학적 일관성 검증
    print("\n5. GEOMETRIC CONSISTENCY VALIDATION")
    print("-" * 40)
    
    geometric_score = 0
    
    if len(successful_poses) >= 2 and len(detection_info) >= 2:
        # 에피폴라 기하학 검증 (단순화된 버전)
        try:
            # 첫 번째와 두 번째 이미지 간의 본질 행렬 계산
            R1, t1 = successful_poses[0][1], successful_poses[0][2]
            R2, t2 = successful_poses[1][1], successful_poses[1][2]
            
            # 상대 포즈 계산
            R_rel = R2 @ R1.T
            t_rel = t2 - R_rel @ t1
            
            # 본질 행렬 계산
            t_skew = np.array([[0, -t_rel[2,0], t_rel[1,0]],
                              [t_rel[2,0], 0, -t_rel[0,0]],
                              [-t_rel[1,0], t_rel[0,0], 0]])
            E = t_skew @ R_rel
            
            # 본질 행렬의 특이값 검사
            U, S, Vt = np.linalg.svd(E)
            singular_ratio = S[1] / S[0] if S[0] > 0 else 0
            
            print(f"Essential matrix singular values: {S}")
            print(f"Singular value ratio: {singular_ratio:.4f}")
            
            # 기하학적 일관성 점수화 (5점 만점)
            if 0.8 < singular_ratio < 1.2:  # 이상적으로는 두 개의 동일한 특이값
                geometric_score = 5
            elif 0.5 < singular_ratio < 1.5:
                geometric_score = 3
            else:
                geometric_score = 1
            
            print(f"Geometric consistency score: {geometric_score}/5 points")
            
        except Exception as e:
            print(f"Could not compute geometric consistency: {e}")
            geometric_score = 2  # 중간 점수
    
    validation_results['geometric'] = {'score': geometric_score}
    validation_results['overall_score'] += geometric_score
    
    # 최종 평가
    print("\n" + "="*60)
    print("OVERALL QUALITY ASSESSMENT")
    print("="*60)
    
    total_score = validation_results['overall_score']
    max_score = validation_results['max_score']
    percentage = (total_score / max_score) * 100
    
    print(f"Total Score: {total_score}/{max_score} ({percentage:.1f}%)")
    
    if percentage >= 90:
        overall_grade = "EXCELLENT"
    elif percentage >= 80:
        overall_grade = "GOOD"
    elif percentage >= 70:
        overall_grade = "ACCEPTABLE"
    elif percentage >= 60:
        overall_grade = "POOR"
    else:
        overall_grade = "UNACCEPTABLE"
    
    print(f"Overall Grade: {overall_grade}")
    
    # 개선 제안
    print(f"\nRECOMMendations:")
    if validation_results.get('reprojection', {}).get('score', 0) < 3:
        print("• Improve image quality and lighting conditions")
        print("• Use more images with better checkerboard detection")
        print("• Check for motion blur or focus issues")
    
    if validation_results.get('intrinsics', {}).get('score', 0) < 4:
        print("• Verify camera setup and lens distortion")
        print("• Use images covering the entire field of view")
    
    if validation_results.get('homography', {}).get('score', 0) < 3:
        print("• Ensure sufficient perspective change between images")
        print("• Avoid nearly parallel checkerboard orientations")
    
    validation_results['overall_grade'] = overall_grade
    validation_results['percentage'] = percentage
    
    return validation_results

def plot_validation_summary(validation_results):
    """
    검증 결과를 시각적으로 요약하는 함수
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 점수 분포 차트
    categories = ['Reprojection', 'Intrinsics', 'Homography', 'Extrinsics', 'Geometric']
    scores = [
        validation_results.get('reprojection', {}).get('score', 0),
        validation_results.get('intrinsics', {}).get('score', 0),
        validation_results.get('homography', {}).get('score', 0),
        validation_results.get('extrinsics', {}).get('score', 0),
        validation_results.get('geometric', {}).get('score', 0)
    ]
    
    colors = ['red' if s < 3 else 'orange' if s < 4 else 'green' for s in scores]
    
    axes[0].bar(categories, scores, color=colors, alpha=0.7, edgecolor='black')
    axes[0].set_ylim(0, 5)
    axes[0].set_ylabel('Score')
    axes[0].set_title('Calibration Quality Scores')
    axes[0].grid(True, alpha=0.3)
    
    # 전체 품질 게이지
    percentage = validation_results.get('percentage', 0)
    
    theta = np.linspace(0, np.pi, 100)
    r = np.ones_like(theta)
    
    axes[1] = plt.subplot(1, 2, 2, projection='polar')
    axes[1].plot(theta, r, 'k-', linewidth=2)
    axes[1].fill_between(theta, 0, r, alpha=0.1)
    
    # 점수에 따른 색상 섹션
    sections = [(0, 60, 'red'), (60, 70, 'orange'), (70, 80, 'yellow'), (80, 90, 'lightgreen'), (90, 100, 'green')]
    
    for start, end, color in sections:
        section_theta = np.linspace(start/100 * np.pi, end/100 * np.pi, 20)
        section_r = np.ones_like(section_theta)
        axes[1].fill_between(section_theta, 0, section_r, color=color, alpha=0.3)
    
    # 현재 점수 표시
    current_theta = percentage/100 * np.pi
    axes[1].plot([current_theta, current_theta], [0, 1], 'k-', linewidth=3)
    axes[1].scatter([current_theta], [0.8], s=200, c='black', marker='o')
    
    axes[1].set_ylim(0, 1)
    axes[1].set_theta_zero_location('W')
    axes[1].set_theta_direction(1)
    axes[1].set_thetagrids(np.arange(0, 181, 30), ['0%', '17%', '33%', '50%', '67%', '83%', '100%'])
    axes[1].set_title(f'Overall Quality: {percentage:.1f}%\n{validation_results.get("overall_grade", "Unknown")}')
    
    plt.tight_layout()
    plt.show()

# 실행 코드
if 'calibration_results' in globals():
    print("Found calibration results from previous cell. Starting validation...")
    validation_results = validate_calibration_comprehensive(calibration_results)
    plot_validation_summary(validation_results)
    
    # 결과 저장
    validation_summary = {
        'overall_score': validation_results['overall_score'],
        'overall_grade': validation_results['overall_grade'],
        'percentage': validation_results['percentage'],
        'reprojection_error': validation_results.get('reprojection', {}).get('rms_error', 0)
    }
    
    print(f"\n📊 VALIDATION SUMMARY:")
    print(f"   Score: {validation_summary['overall_score']}/25")
    print(f"   Grade: {validation_summary['overall_grade']}")
    print(f"   Percentage: {validation_summary['percentage']:.1f}%")
    print(f"   RMS Error: {validation_summary['reprojection_error']:.4f} pixels")
    
else:
    print("❌ Error: 'calibration_results' not found.")
    print("Please run the calibration code in the previous cell first.")